### 1. 선형화귀분석

#### 예제: 데이터의 sqft_living(주거공간의 평방피트)를 독립변수, price를 종속변수로 설정하여 단순선형회귀분석을 실시한 후, 추정되는 회귀모형에 대해 해석하라

In [ ]:
import pandas as pd
import numpy as np
f_path = 
data = pd.read_csv(f_path)
data.head()

In [ ]:
col_y = 
col_x = 

In [ ]:
data = data[[col_y, col_x]]
data.corr()

In [ ]:
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
#변수할당
y = data[col_y]
x = data[[col_x]]
#단순 선형회귀 모형 적합
lr = ols(f'{col_y}~{col_x}', data=data).fit()
y_pred = lr.predict(x)
#시각화
plt.scatter(x, y)
plt.plot(x,y_pred, color='red')
plt.xlabel(f'{col_x}', fontsize=10)
plt.ylabel(f'{col_y}', fontsize=10)
plt.title('Linear Regression Result')
plt.show()

In [ ]:
'''
Prob(F-statistic) 값이 F통계량으로
귀무가설 : 회귀모형은 유의하지 않다
대립가설 : 회귀모형은 유의하다
'''
lr.summary()

In [ ]:
if lr.f_pvalue < 0.05:
    print('대립가설 채택, 회귀모형은 유의하다')
    print(f'결정계수 R2은 {lr.rsquared}이다.')
    print(f'결론, 전체 데이터의 {lr.rsquared*100}%를 설명하는 회귀식을 구할 수 있다.')
    #lr.summary()의 coef열의 intercept 값은 상수값, col_x명의 값은 기울기 값으로 사용 가능하다.
else:
    print('귀무가설 채택, 회귀모형은 유의하지 않다.')

### 2. 다중 선형회귀분석

#### 예제: 다중 선형회귀분석에서 다중공선성ㅇㄹ 제거하고 최적의 모델을 찾기 위해 변수선택법을 적용하여 진행한다.

In [ ]:
import pandas as pd
f_path = 
data = pd.read_csv(f_path)
data.info()

In [ ]:
col_y = 
col_x1 = 
col_x2 = 
col_x3 = 
col_x4 = 

In [ ]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
#ols모델 formula을 정의할 때, 일부 특수문자는 쓸수 없기 때문에, 컬럼 특수문자 제거
data.columns = data.columns.str.replace(".","")
model = smf.ols(formula=f'{col_y}~{col_x1}+{col_x2}+{col_x3}+{col_x4}', data = data)
result = model.fit()
result.summary()
'''
1. Adf.R-squared값을 통해, 전체 데이터의 R2*100 % 설명가능함을 확인
2. Prob(F-statistic)값을 통해, p-value로 0.05 이하 여부를 확인 후 이하이면, 회귀모형 유의 함
3. (P>|t|) 값이 p-value 유의수준보다 높은 컬럼명을 제거 한 후 다시 모델을 돌려보는 방안 
'''

In [ ]:
# 다중공선성 파악을 위한 상관관계 확인
data[[col_x1, col_x2, col_x3, col_x4]].corr()
'''
플러스/마이너스 포함하여 0.9이상인 값의 다중공선성 변수를 확인 한다.
'''

In [ ]:
# VIF값을 구해보자
'''
vif = 1/(1-r2)로 VIF가 10 이상인 경우 일반적으로 다중공선성이 있는 변수라고 판단
'''
from patsy import dmatrices
from statsmodels.stats.outliers_infuence import variance_inflation_factor
#독립변수와 종속변수를 데이터프레임으로 나누어 저장하는 함수
y, x = dmatrices(formula=f'{col_y}~{col_x1}+{col_x2}+{col_x3}+{col_x4}', data = data, return_type='dataframe')
#독립변수끼리의 vif값을 계산하여 데이터프레임으로 만드는 과정
vif_list=[]
for i in range(1,len(x.columns)):
    vif_list.append([variance_inflation_factor(x.values, i), x.columns[i])
pd.DataFrame(vif_list, columns=['vif', 'variable'])
'''
vif가 가장 높은 variable인 컬러명을 제거대상으로 함
'''

In [ ]:
# 모델 summary()를 다시 확인해 본다.
model = smf.ols(formula = f'{col_y}~{col_x1}+{col_x2}+{col_x3}', data=data)
result = model.fit()
result.summary()
'''
1. Adf.R-squared값을 통해, 전체 데이터의 R2*100 % 설명가능함을 확인
2. Prob(F-statistic)값을 통해, p-value로 0.05 이하 여부를 확인 후 이하이면, 회귀모형 유의 함
3. (P>|t|) 값이 p-value 유의수준보다 높은 컬럼명을 제거 한 후 다시 모델을 돌려보는 방안 
'''

In [ ]:
# 독립변수 중 유이한 변수를 고르고, 모델의 성능을 최적화시키는 변수선택법을 진행
import time
import itertools
def processSubset(x,y,feature_set):
'''
AIC는 모델의 성능지표로서 MSE에 변수 수만큼 penalty를 주는 것으로 AIC 값이 낮을 수록 모델 적합도가 좋다
'''
    model = sm.OLS(y,x[list(feature_set)]) #Modeling
    regr = model.fit() 
    AIC = regr.aic # 모델의 AIC
    return {"model":regr, "aic":AIC}

#전진선택법
def forward(x,y,predictors):
    col_x='Intercept'
    #데이터 변수들이 미리 정의된 predictors에 있는지 없는지 확인 및 분류
    remaining_predictors = [p for p in x.columns.difference([col_x])
                            if p not in predictors]
    results = []
    for p in remainig_predictors:
        results.append(precessSubset(x=x,y=y,feature_set=predictors+[p]+[col_x]))
    #데이터프레임으로 변환
    models=pd.DataFrame(results)
    #AIC가 가장 낮은 것을 선택
    best_model = models.loc[models['AIC'].argmin()]#index
    print('Processed', models.shape[0], 'models on', len(predictors)+1, 'predictors in')
    print('Selected predictors:', best_model['model'].model.exog_names, 'AIC:', best_model[0])
    return best_model

#후진소거법
def backward(x,y,predictors):
    col_x='Intercept'
    tic=time.time()
    results=[]
    #데이터 변수들이 미리 정의된 predictors 조합 확인
    for combo in itertools.combinations(predictors, len(predictors) -1):
        results.append(processSubset(x=x, y=y, feature_set=list(combo)+[col_x]))
    #데이터프레임으로 변환
    models=pd.DataFrame(results)
    #가장 낮은 AIC를 가진 모델을 선택
    best_model = models.loc[models['AIC'].argmin()]
    toc = time.time()
    print('Processed', models.shape[0], 'models on',
          len(predictors) -1, 'predictors in', (toc-tic))
    print('Selected predictors:', best_model['model'].model.exog_names, 'AIC:', best_model[0])
    return best_model
    
#단계적 선택법
def Stepwise_model(x,y):
    stepmodels = pd.DataFrame(columns=['AIC', "model"])
    tic=time.time()
    predictors = []
    smodel_before = processSubset(x,y,predictors+['Intercept'])['AIC']
    for i in range(1, len(x.columns.difference(['Intercept']))+1):
        forward_result=forward(x=x, y=y, predictors=predictors)
        print('forward')
        stepmodels.loc[i]=forward_result
        predictors = stemodels.loc[i]['mdoel'].model.exog_names
        predictors = [k for k in predictors if k!='Intercept']
        Backward_result = bacward(x=x, y=y, predictors=predictors)

        if Backward_result['Aic']<Forward_result['AIC']:
            stepmodels.loc[i] = Backward_result
            predictors = stepmodels.loc[i]['model'].model.xeog_names
            smodel_before = stepmodels.loc[i]['AIC']
            predictors = [k for k in predictors if k='Intercept']
            print('backward')

        if stepmodels.loc[i]['AIC']>smodel_before:
            break
        else:
            smodel_before = stepmodels.loc[i]['AIC']
    toc = time.time()
    print('total elapsed time: ', (toc-tic), 'seconds.')
    return (stepmodels['model'][len(stepmodels['model'])])

stepwise_best_model = stepwise_model(x=x, y=y)


stepwise_best_model.summary()